In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import re
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

In [2]:
from tqdm import tqdm


In [3]:
# Load data using relative paths
raw_data = pd.read_csv("C:/Users/e.sansebastian\OneDrive - EXPONDO/Ironhack/Weeks/Week_9/Final_project/final_project/data/data_uncleaned.csv")
cleaned_data = pd.read_csv("C:/Users/e.sansebastian\OneDrive - EXPONDO/Ironhack/Weeks/Week_9/Final_project/final_project/data/df_premodel.csv",sep = ";")
resampled_data = pd.read_csv("C:/Users/e.sansebastian\OneDrive - EXPONDO/Ironhack/Weeks/Week_9/Final_project/final_project/data/data_resampled.csv")

In [4]:
# We create review ids out of the indexes to be able to concatenate the df with the resulst of the modelling
raw_data['review_id'] = raw_data.index
cleaned_data['review_id'] = cleaned_data.index
resampled_data['review_id'] = resampled_data.index

In [5]:
raw_data["review_rating"] = raw_data["review_rating"].astype(str)
cleaned_data["review_rating"] = cleaned_data["review_rating"].astype(str)
resampled_data["review_rating"] = resampled_data["review_rating"].astype(str)
raw_data['review_id'] = raw_data['review_id'].astype(str)
cleaned_data['review_id'] = cleaned_data['review_id'].astype(str)
resampled_data['review_id'] = resampled_data['review_id'].astype(str)
display(raw_data.head())
display(cleaned_data.head())
display(resampled_data.head())

,Unnamed: 0,review_title,review_rating,review_content,source,review_id
0,0,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo,0
1,1,Awesome experience,5,Awesome experience. We had to use their custom...,expondo,1
2,2,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo,2
3,3,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo,3
4,4,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo,4


,review_title,review_rating,review_content,source,review_id
0,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,0
1,awesome experience,5,awesome experience we had to use their custome...,expondo,1
2,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,2
3,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,3
4,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,4


,review_title,review_rating,review_content,source,sentiment,review_id
0,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,positive,0
1,awesome experience,5,awesome experience we had to use their custome...,expondo,positive,1
2,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,positive,2
3,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,positive,3
4,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,negative,4


In [6]:
# We drop the first column "Unnamed: 0" from the raw_data df:
raw_data.drop(columns = "Unnamed: 0",inplace = True)
raw_data['rating_sentiment'] = raw_data['review_rating'].apply(lambda x: 'negative' if x in ['1', '2'] else ('neutral' if x == '3' else 'positive'))
cleaned_data['rating_sentiment'] = cleaned_data['review_rating'].apply(lambda x: 'negative' if x in ['1', '2'] else ('neutral' if x == '3' else 'positive'))
resampled_data['rating_sentiment'] = resampled_data['review_rating'].apply(lambda x: 'negative' if x in ['1', '2'] else ('neutral' if x == '3' else 'positive'))

In [7]:
display(raw_data.dtypes)
display(cleaned_data.dtypes)
display(resampled_data.dtypes)

review_title        object
review_rating       object
review_content      object
source              object
review_id           object
rating_sentiment    object
dtype: object

review_title        object
review_rating       object
review_content      object
source              object
review_id           object
rating_sentiment    object
dtype: object

review_title        object
review_rating       object
review_content      object
source              object
sentiment           object
review_id           object
rating_sentiment    object
dtype: object

In [8]:
raw_data.head()

,review_title,review_rating,review_content,source,review_id,rating_sentiment
0,"TIG (WIG) welder, model S-WIGMA 200PDC",5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",expondo,0,positive
1,Awesome experience,5,Awesome experience. We had to use their custom...,expondo,1,positive
2,The product arrived on time all the…,5,The product arrived on time all the assembly d...,expondo,2,positive
3,Good and fast delivery,5,Good and fast delivery of this amazing standin...,expondo,3,positive
4,Bad invoice cross-border purchase - no contact...,1,Bad invoice - DPH on invoice - shipping to slo...,expondo,4,negative


In [9]:
resampled_data.shape

(15375, 7)

## Sentiment analysis

### Step 1. VADER Seniment Scoring

Step 1. VADER Sentiment Scoring
We will use NLTK's SentimentIntensityAnalyzer to get the neg/neu/pos scores of the text.

This uses a "bag of words" approach:
Stop words are removed
each word is scored and combined to a total score.

In [10]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [11]:
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(resampled_data.iterrows(), total=len(resampled_data)):
    text = row['review_content'] if not pd.isna(row['review_content']) else ''  # Convert to string, handle NaN
    myid = row['review_id']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/15375 [00:00<?, ?it/s]

In [12]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'id'})
display(vaders.shape)
display(vaders.head())

(15375, 5)

,id,neg,neu,pos,compound
0,0,0.029,0.847,0.124,0.9300
1,1,0.000,0.557,0.443,0.9260
2,2,0.000,0.571,0.429,0.9147
3,3,0.000,0.593,0.407,0.8834
4,4,0.390,0.530,0.080,-0.7920


In [13]:
display(resampled_data.shape)
result_df_resample = vaders.merge(resampled_data, how='left',left_on="id",right_on="review_id")
display(result_df_resample.head())
result_df_resample.shape

(15375, 7)

,id,neg,neu,pos,compound,review_title,review_rating,review_content,source,sentiment,review_id,rating_sentiment
0,0,0.029,0.847,0.124,0.9300,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,positive,0,positive
1,1,0.000,0.557,0.443,0.9260,awesome experience,5,awesome experience we had to use their custome...,expondo,positive,1,positive
2,2,0.000,0.571,0.429,0.9147,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,positive,2,positive
3,3,0.000,0.593,0.407,0.8834,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,positive,3,positive
4,4,0.390,0.530,0.080,-0.7920,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,negative,4,negative


(15375, 12)

In [14]:
result_df_resample["sentiment_vader"] = result_df_resample["compound"].apply(lambda x: 'positive' if x >= 0.05 else ('negative' if x <= -0.05 else 'neutral'))

In [15]:
result_df_resample.head()

,id,neg,neu,pos,compound,review_title,review_rating,review_content,source,sentiment,review_id,rating_sentiment,sentiment_vader
0,0,0.029,0.847,0.124,0.9300,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,positive,0,positive,positive
1,1,0.000,0.557,0.443,0.9260,awesome experience,5,awesome experience we had to use their custome...,expondo,positive,1,positive,positive
2,2,0.000,0.571,0.429,0.9147,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,positive,2,positive,positive
3,3,0.000,0.593,0.407,0.8834,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,positive,3,positive,positive
4,4,0.390,0.530,0.080,-0.7920,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,negative,4,negative,negative


## Step 2. TextBlob

In [47]:
result_df_resample.head()

,id,neg,neu,pos,compound,review_title,review_rating,review_content,source,sentiment,review_id,rating_sentiment,sentiment_vader,roberta_neg,roberta_neu,roberta_pos,sentiment_textblob,roberta_sentiment
0,0,0.029,0.847,0.124,0.9300,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,positive,0,positive,positive,0.025944,0.220876,0.753179,positive,positive
1,1,0.000,0.557,0.443,0.9260,awesome experience,5,awesome experience we had to use their custome...,expondo,positive,1,positive,positive,0.008892,0.049066,0.942042,positive,positive
2,2,0.000,0.571,0.429,0.9147,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,positive,2,positive,positive,0.002580,0.015203,0.982217,positive,positive
3,3,0.000,0.593,0.407,0.8834,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,positive,3,positive,positive,0.001689,0.015697,0.982614,positive,positive
4,4,0.390,0.530,0.080,-0.7920,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,negative,4,negative,negative,0.685091,0.293279,0.021630,negative,negative


In [48]:
from textblob import TextBlob

In [49]:
def analyze_sentiment(text):
    analysis = TextBlob(text)
    # Classify polarity as positive, negative, or neutral
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [50]:
# Apply sentiment analysis to the 'Review Content' column
result_df_resample['sentiment_textblob'] = result_df_resample['review_content'].astype(str).apply(analyze_sentiment)

In [51]:
# Display the resulting DataFrame
display(result_df_resample.head())

,id,neg,neu,pos,compound,review_title,review_rating,review_content,source,sentiment,review_id,rating_sentiment,sentiment_vader,roberta_neg,roberta_neu,roberta_pos,sentiment_textblob,roberta_sentiment
0,0,0.029,0.847,0.124,0.9300,tig wig welder model s wigma pdc,5,i ordered a tig wig mma welder model s wigma p...,expondo,positive,0,positive,positive,0.025944,0.220876,0.753179,positive,positive
1,1,0.000,0.557,0.443,0.9260,awesome experience,5,awesome experience we had to use their custome...,expondo,positive,1,positive,positive,0.008892,0.049066,0.942042,positive,positive
2,2,0.000,0.571,0.429,0.9147,the product arrived on time all the,5,the product arrived on time all the assembly d...,expondo,positive,2,positive,positive,0.002580,0.015203,0.982217,positive,positive
3,3,0.000,0.593,0.407,0.8834,good and fast delivery,5,good and fast delivery of this amazing standin...,expondo,positive,3,positive,positive,0.001689,0.015697,0.982614,positive,positive
4,4,0.390,0.530,0.080,-0.7920,bad invoice cross border purchase no contact t...,1,bad invoice dph on invoice shipping to slovaki...,expondo,negative,4,negative,negative,0.685091,0.293279,0.021630,negative,negative


In [52]:
result_df_resample.to_csv("result_df_resample.csv",index=False,sep=";")

In [53]:
#results_df.to_csv("results_for sentiment2.csv",index=False,sep=";")

In [54]:
from sklearn.metrics import classification_report

print("TextBlob:")
print(classification_report(result_df_resample['sentiment'], result_df_resample['sentiment_textblob']))

print("\nVADER:")
print(classification_report(result_df_resample['sentiment'], result_df_resample['sentiment_vader']))

#print("\nRoBERTa:")
#print(classification_report(result_df_resample['sentiment'], result_df_resample['roberta_sentiment']))


TextBlob:
              precision    recall  f1-score   support

    negative       0.63      0.46      0.53      5125
     neutral       0.44      0.09      0.15      5125
    positive       0.45      0.92      0.60      5125

    accuracy                           0.49     15375
   macro avg       0.50      0.49      0.43     15375
weighted avg       0.50      0.49      0.43     15375


VADER:
              precision    recall  f1-score   support

    negative       0.59      0.51      0.55      5125
     neutral       0.49      0.09      0.16      5125
    positive       0.47      0.91      0.62      5125

    accuracy                           0.51     15375
   macro avg       0.52      0.51      0.44     15375
weighted avg       0.52      0.51      0.44     15375

